In [27]:
# Change the path if necessary
datasets_path = 'datasets'

In [29]:
import sys
import os

In [28]:
# Deploy AML code in colab
if 'google.colab' in sys.modules:
  !wget -q https://raw.githubusercontent.com/tsunrise/colab-github/main/colab_github.py
  import colab_github
  colab_github.github_auth(persistent_key=False)

  # Remember to follow the instructions to add the SSH key to your account in order to clone private repo
  !git clone git@github.com:oz-e/applied-ml.git
  if os.path.exists('/content/applied-ml/'):
    %cd /content/applied-ml/
  else:
    raise Exception('Please follow the instructions to add the SSH key to your account in order to clone private repo')

  # Install any other requirements (to be converted to requirements.txt)
  !pip install openai-clip

Looks that a private key is already created. If you have already push it to github, no action required.
 Otherwise, Please go to https://github.com/settings/ssh/new to upload the following key: 
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIPl/C4mu70CgpXdJQR8RM/n9ZJybh+BWphoQqy9o0inv root@4c0f2d2c466d

Please use SSH method to clone repo.
fatal: destination path 'applied-ml' already exists and is not an empty directory.
/content/applied-ml


In [ ]:
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm

In [30]:
model, preprocess = clip.load("ViT-B/16")

In [31]:
test_templates = [
    'a photo of a {}.',
]

In [32]:
import aml.datasets

caltech101_dataset_test = aml.datasets.Caltech101(datasets_path, split='test', transform=preprocess)
loader = torch.utils.data.DataLoader(caltech101_dataset_test, batch_size=32, num_workers=2)

In [33]:
def zeroshot_classifier(classnames, templates):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(classnames):
            texts = [template.format(classname) for template in templates] #format with class
            texts = clip.tokenize(texts).cuda() #tokenize
            class_embeddings = model.encode_text(texts) #embed with text encoder
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
    return zeroshot_weights


zeroshot_weights = zeroshot_classifier(caltech101_dataset_test.classnames, test_templates)

  0%|          | 0/100 [00:00<?, ?it/s]

In [34]:
def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

In [36]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()

        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

  0%|          | 0/78 [00:00<?, ?it/s]

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/applied-ml/aml/datasets.py", line 63, in __getitem__
    img = self.transform(img)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torchvision/transforms/transforms.py", line 95, in __call__
    im